# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import glob
import logging

from astropy import constants as const
from astropy import units as u

import cmasher as cmr

from colossus.cosmology import cosmology
from colossus.halo import mass_defs
from colossus.lss import mass_function

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import matplotlib.colors as mp_colors
import matplotlib.cm as cmx

import numpy as np

import pandas as pd
pd.set_option('max_columns', None)

import mockUniv.calc_rad_paper.fun_sum_area as fun_sum_area #fun_distrib_paper as calc_distrib
import mockUniv.calc_rad_paper.calc as calc
import mockUniv.calc_rad_paper.fun_significance as fun_significance

# Read data 

The light cones are going to be stores in a dictionary. In the next block of code we define the keys to access the different cluster ids (`7830644447` and `7793510527`), the light cones type either cluster (`c_rand`) or field (`f_rand`), the name of the observed clusters corresponding to the different area curves and the energy bands of the observations. In the paper we only use full band but in the data release we also provide hard and soft bands, you can include them in the variable `bands`. 

We also define the cosmology (needed to calculate masses conversions), and the redshift of the clusters.

In [ ]:
id_lst = ['7830644447', '7793510527'] 

lc_type_lst = ['c_rand', 'f_rand']

date_lst = {'7830644447':['20231123', '20231123'],
            '7793510527':['20231121', '20231121'],
}

desire_redshift = 0.972    
cosmo = cosmology.setCosmology('planck18')

clustname_lst = ['SPT-CLJ2146-4633', 'PLCKG266.6-27.3']
corename = ['13469','pl26'] 
bands = ['full']

We load the light cones, you can download the data form [Zenodo](https://zenodo.org/uploads/11446317). **Remember to change variable `pth` to your location of the data**

In [ ]:
pth = './input/lc_rad_dst/'

num_obs = 100

lc_sample_list_csv = {}

for lc_list in id_lst:
    lc_sample_list_csv.update({ f'{lc_list}':{} })
    print(lc_list)
    for loop, lc_type in enumerate(['c_rand', 'f_rand']):
        lc_sample_list_csv[lc_list].update({f'{lc_type}':[] })
        kk = lc_type.split('_')[0]
        strng = kk + f'_{date_lst[lc_list][loop]}'

        for lc_pth in sorted(glob.glob(pth + f'{lc_list}/{strng}/lc_???_drop.h5')):
            lc = pd.read_hdf(lc_pth, 'df')
            lc_sample_list_csv[lc_list][lc_type].append(lc)
            print(f'\t{lc_pth}')

We read the observational data points 

In [ ]:
rad_distrib_obs = {}
for loop_clust, clust_name in enumerate(clustname_lst):
    for energy_band in bands:  
        file = f'./input/rad_dst_obs/rad_distrib_{clustname_lst[loop_clust]}_{energy_band}.dat'
        print(clust_name, file)
        rad_distrib_obs.update({ f'{corename[loop_clust]}_{energy_band}': pd.read_csv(file, sep=';') })

# Annulus def.

The following block of code define $R_{500}$ corresponding to the different clusters in **physical Mpc**.

In [ ]:
r_dict = {}

for id_obj in id_lst:
    print(id_obj)
    m_id = lc_sample_list_csv[f'{id_obj}']['c_rand'][0]['id'] == int(id_obj)
    cvir = 10.0**(0.7) 
    Mvir = float(10**lc_sample_list_csv[f'{id_obj}']['c_rand'][0][m_id]['lgm'])
    print('lgM_vir=', np.log10(Mvir))
    print('R_vir (comoving kpc/h)=', float(lc_sample_list_csv[f'{id_obj}']['c_rand'][0][m_id]['r']) )
    M500c, R500c, c500c = mass_defs.changeMassDefinition(Mvir, 
                                                         cvir, 
                                                         desire_redshift,
                                                         'vir', 
                                                         '500c')
    print( 'lgM_500=', np.log10(M500c) )
    print( 'R_500 (phys kpc/h)=', R500c ) 
    
    r_dict.update({ f'{id_obj}': R500c*u.kpc.to(u.Mpc)/cosmo.h})
    print()
r_dict

def get_annuli_rad(r, desire_z, cosmo):
    """
    Function that calculates the anulii in angular size for a given radius.
    """
    annulus_radius = r * np.arange(0, 5.5, 0.5) / cosmo.angularDiameterDistance(desire_z)*u.rad
    return annulus_radius

We transform the physical size of the cluster to angular size and we define the different anulii

In [ ]:
annulus_radius_dict = {}

for id_obj in id_lst: 
    annulus_radius = calc.get_annuli_rad(
        r_dict[id_obj] * cosmo.h,
        desire_redshift,
        cosmo
    )
    annulus_radius_dict.update({f'{id_obj}': annulus_radius})
annulus_radius_dict[id_lst[0]].to(u.arcmin)

# Summed area

Sum the area of individual objects per ring $\rightarrow$ total area per ring per lc

In [ ]:
sum_area_model_dict_ids = {}
models = ['AMedian', 'GMedian'] 
models_f = ['AMedian', 'GMedian']

for id_obj in id_lst:
    print(f'{id_obj}')
    sum_area_model_dict_ids.update(
        {f'{id_obj}': fun_sum_area.sum_area_ids(corename, 
                                   bands, 
                                   lc_sample_list_csv[f'{id_obj}'],
                                   annulus_radius_dict[id_obj], 
                                   models,
                                   models_f=models_f
                    )
        }
)

In [ ]:
num_obs = 100

sum_area_field_model_suffle_dict_ids = {}
for id_obj in id_lst: 
    print(f'{id_obj}')
    sum_area_field_model_suffle_dict_ids.update( 
                             {f'{id_obj}': fun_sum_area.get_sum_area_field_suffle_ids(
                                 num_obs, corename, bands, sum_area_model_dict_ids[f'{id_obj}'], models_f
                                ) 
                             }
)    

# Peak signif.

In [ ]:
xplot = (np.arange(0, 7.5, .5)[1:] + np.arange(0, 7.5, .5)[:-1])*.5

lc_type = 'c_rand'
lc_type_f = 'f_rand'

sum_area_model_dict_bkgSubs_ids = {}
sum_area_model_dict_bkgSubs_indiv_ids = {}
oneSigmaUp_sum_area_model_dict_bkgSubs_ids = {}
oneSigmaLow_sum_area_model_dict_bkgSubs_ids = {}

dict_lst = [ sum_area_model_dict_bkgSubs_ids, 
             sum_area_model_dict_bkgSubs_indiv_ids, 
             oneSigmaUp_sum_area_model_dict_bkgSubs_ids,
             oneSigmaLow_sum_area_model_dict_bkgSubs_ids ]

for id_obj in id_lst:
    print(f'{id_obj}')
    out = fun_sum_area.get_sum_bkg_subs_and_ci(
                                               xplot,
                                               lc_type, 
                                               lc_type_f, 
                                               corename,
                                               bands, 
                                               sum_area_model_dict_ids[id_obj], 
                                               sum_area_field_model_suffle_dict_ids[id_obj],
                                               l_lim=0.16,
                                               u_lim=0.84,
                                              )
    for loop, elem in enumerate(out):
        dict_lst[loop].update({f'{id_obj}': elem})

# Mean radial distribution

In [ ]:
for id_obj in id_lst[:1]:
    print(id_obj)
    m_id = lc_sample_list_csv[f'{id_obj}']['c_rand'][0]['id'] == int(id_obj)
    cvir = 10.0**(0.7) 
    Mvir = float(10**lc_sample_list_csv[f'{id_obj}']['c_rand'][0][m_id]['lgm'])
    print('lgM_vir=', np.log10(Mvir))
    print('R_vir (comoving kpc/h)=',float(lc_sample_list_csv[f'{id_obj}']['c_rand'][0][m_id]['r']) )
    M200c, R200c, c200c = mass_defs.changeMassDefinition(Mvir, 
                                                         cvir, 
                                                         desire_redshift,
                                                         'vir', 
                                                         '200c')
    M200c, R500c, c200c = mass_defs.changeMassDefinition(Mvir, 
                                                         cvir, 
                                                         desire_redshift,
                                                         'vir', 
                                                         '500c')
    
    R500c = R500c/cosmo.h
    R200c = R200c/cosmo.h
    rvir = float(lc_sample_list_csv[f'{id_obj}']['c_rand'][0][m_id]['r'])/cosmo.h/(1+0.97)

In [ ]:
with plt.style.context('default'):
    fig = plt.figure(figsize=(6, 5), dpi=150)
    ax = fig.add_subplot(111)
    
    xplot = (np.arange(0, 5.5, .5)[1:] + np.arange(0, 5.5, .5)[:-1])*.5
    
    color_model = ['tab:orange', 'tab:green']
    markers = ['o', 's', '^']
    model_names  = ['Model 1', 'Model 2']
    jet = cm = plt.get_cmap('plasma') 
    values = np.arange(0, 10, 1)
    cNorm  = mp_colors.Normalize(vmin=0, vmax=values[-1])
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
    
    ax.fill_between([], [],
                    alpha=0.5,
                    color='k',
                    label='86% c.i.'
    )
    
    id_clust = id_lst[0]
    name_clust = 'pl26'
    energy_band = 'full'
    lc_type = 'c_rand'
    lc_type_f = 'f_rand'
    
    ax.errorbar(rad_distrib_obs[f'{name_clust}_{energy_band}'].anulii[:-1], 
                rad_distrib_obs[f'{name_clust}_{energy_band}'].Nagn[:-1],
                yerr=rad_distrib_obs[f'{name_clust}_{energy_band}'].e_Nagn[:-1],
                marker='s',
                capsize=6,
                ms=8,
                color='k',
                label='Observations'
               )
    for loop_model, sum_area_loop in enumerate(sum_area_model_dict_ids[id_clust][name_clust][energy_band][lc_type][:2]):
        ax.fill_between(xplot[:8],
                        np.array(oneSigmaLow_sum_area_model_dict_bkgSubs_ids[id_clust][f'{name_clust}'][f'{energy_band}'][loop_model][:8]),
                        np.array(oneSigmaUp_sum_area_model_dict_bkgSubs_ids[id_clust][f'{name_clust}'][f'{energy_band}'][loop_model][:8]),
                        color=color_model[loop_model],
                        alpha=.25)
        
        ax.plot(xplot[:8], 
                sum_area_model_dict_bkgSubs_ids[id_clust][f'{name_clust}'][f'{energy_band}'][loop_model][:8],
                marker=markers[loop_model],
                lw=3,
                ms=0,
                color=color_model[loop_model],
                label=f'{model_names[loop_model]}',
                )
        
    ax.plot([0,5],[0,0],
           ls=':',
           lw=1.5,
           color='k')
    ax.plot([R200c/R500c, R200c/R500c],[-20, 50],
           ls='--',
           lw=1.5,
           color='k')
    ax.plot([rvir/R500c, rvir/R500c],[-20, 50],
           ls='-.',
           lw=1.5,
           color='k')

    ax.text(R200c/R500c-0.15, 18, '$R_{200c}$/$R_{500}$',rotation='vertical')
    ax.text(rvir/R500c+0.075, 18, '$R_{vir}$/$R_{500}$',rotation='vertical')
    
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    
    
    ax.set_xlabel('r/R$_{500}$', fontsize=16)
    ax.set_ylabel('Number AGN', fontsize=16)
    
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    
    ax.legend(loc='upper right')
    
    ax.set_xlim(0, 4)
    ax.set_ylim(-10, 25)
    ax.grid(True)
    ax.set_title(f'Cluster id={id_clust}', fontsize=16)
    
    fig.tight_layout()
    plt.show()

In [ ]:
with plt.style.context('default'):
    nrows = 2
    ncols = 2
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=[18, 12]) #4, 3
    
    xplot = (np.arange(0, 5.5, .5)[1:] + np.arange(0, 5.5, .5)[:-1])*.5
    
    color_model = ['tab:orange', 'tab:green']
    model_names  = ['Model 1', 'Model 2'] 
    markers = ['o', 's', '^']
    energy_band = 'full'
    lc_type = 'c_rand'
    lc_type_f = 'f_rand'
    
    fs = 26

    ax[0,0].fill_between([], [],
                         alpha=0.5,
                         color='k',
                         label='68% c.i.'
                         )
    for row in range(nrows):
        if row == 0:
            id_clust = id_lst[0]
        else:
            id_clust = id_lst[1]
            
        for col in range(ncols):
            if col == 0 :
                name_clust = '13469'
            else:
                name_clust = 'pl26'
    
            ax[row, col].errorbar(rad_distrib_obs[f'{name_clust}_{energy_band}'].anulii[:-1], 
                        rad_distrib_obs[f'{name_clust}_{energy_band}'].Nagn[:-1],
                        yerr=rad_distrib_obs[f'{name_clust}_{energy_band}'].e_Nagn[:-1],
                        marker='s',
                        capsize=10,
                        ms=16,
                        color='k',
                        label='Observations'
                       )
            for loop_model, sum_area_loop in enumerate(sum_area_model_dict_ids[id_clust][name_clust][energy_band][lc_type][:2]):
                ax[row, col].fill_between(xplot[:8],
                                oneSigmaLow_sum_area_model_dict_bkgSubs_ids[id_clust][f'{name_clust}'][f'{energy_band}'][loop_model][:8],
                                oneSigmaUp_sum_area_model_dict_bkgSubs_ids[id_clust][f'{name_clust}'][f'{energy_band}'][loop_model][:8],
                                color=color_model[loop_model],
                                alpha=.25)
                
                ax[row, col].plot(xplot[:8], 
                        sum_area_model_dict_bkgSubs_ids[id_clust][f'{name_clust}'][f'{energy_band}'][loop_model][:8],
                        marker=markers[loop_model],
                        lw=3,
                        ms=0,
                        color=color_model[loop_model],
                        label=f'{model_names[loop_model]}',
                        )
                
            ax[row, col].plot([0,5],[0,0],
                   ls=':',
                   lw=1.5,
                   color='k')
            
            ax[row, col].tick_params(axis='x', labelsize=fs)
            ax[row, col].tick_params(axis='y', labelsize=fs)
            
            if row != 0:
                ax[row, col].set_xlabel('r/R$_{500}$', fontsize=fs+4)
            else:
                ax[row, col].set(xticklabels=[])
            if col == 0: 
                ax[row, col].set_ylabel(f'Cluster id={id_clust}\nNumber AGN', fontsize=fs)
            else:
                ax[row, col].set(yticklabels=[])
            
            ax[row, col].tick_params(axis='x', labelsize=fs-2)
            ax[row, col].tick_params(axis='y', labelsize=fs-2)
            
            if np.logical_and(row==0, col==0):
                ax[row, col].legend(fontsize=fs-2)
            
            ax[row, col].set_xlim(0, 4)
            ax[row, col].set_ylim(-10, 25)
            ax[row, col].grid(True)
            
            if row == 0:
                if col == 0:
                    name_ttle = clustname_lst[0]
                else:
                    name_ttle = clustname_lst[1]
            
                ax[row, col].set_title(f'{name_ttle}', fontsize=fs+6) #, sensitivity={name_clust}, band={energy_band}\n
    
    fig.tight_layout()
    plt.show()

# Individual radial distributions

In [ ]:
significance = {}

ring_ovrdnst = 4
sigma_lim = 1
sigma_ring_lim = .5


for id_obj in id_lst[:]:
    significance.update( { f'{id_obj}': fun_significance.get_peak_signif(
        ring_ovrdnst, 
        corename,
        bands, 
        oneSigmaUp_sum_area_model_dict_bkgSubs_ids[id_obj], 
        sum_area_model_dict_bkgSubs_indiv_ids[id_obj],
        sigma_lim,
        sigma_ring_lim
    ) } )

In [ ]:
lc_w_significant_peak = np.arange(0, 100, 1)[ significance[id_lst[0]][f'{name_clust}'][f'{energy_band}'][1] ]
lc_w_significant_peak

In [ ]:
nrows = 10
ncols = 10

with plt.style.context('default'):
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=[30, 30])
    
    xplot = (np.arange(0, 7.5, .5)[1:] + np.arange(0, 7.5, .5)[:-1])*.5
    
    lc_counter = 0
    color_model = ['tab:orange', 'tab:green']
    n_lc = 99
    
    name_clust = 'pl26'
    energy_band = 'full'
    lc_type = 'c_rand'
    lc_type_f = 'f_rand'
    
    id_clust = id_lst[0]
        
    for row  in range(0, nrows):
        if lc_counter>n_lc:
                break
        for column in range(0, ncols):
            if lc_counter>n_lc:
                break
            
            for loop_model_, sum_area_loop in enumerate(
                sum_area_model_dict_ids[id_clust][name_clust][energy_band][lc_type][:n_lc][:1]
            ):
                loop_model = -1
                
                ax[row, column].set_title('lc={}'.format(lc_counter), fontsize=25)
                ax[row, column].plot(
                                     xplot,
                                     sum_area_model_dict_bkgSubs_indiv_ids[id_clust][name_clust][energy_band][loop_model][lc_counter],
                                     marker='o',
                                     lw=4,
                                     ls='-',
                                     ms=0,
                                     alpha=0.6,
                                     label='{}'.format(models[loop_model]),
                                     color=color_model[loop_model],
                                     rasterized=True
                                 )
                ax[row, column].errorbar(rad_distrib_obs[f'{name_clust}_{energy_band}'].anulii[:-1], 
                                         rad_distrib_obs[f'{name_clust}_{energy_band}'].Nagn[:-1],
                                         yerr=rad_distrib_obs[f'{name_clust}_{energy_band}'].e_Nagn[:-1],
                                         marker='s',
                                         capsize=6,
                                         ms=8,
                                         alpha=0.1,
                                         color='k',
                                         label='Observations',
                                         rasterized=True
                                         )
                
                if np.isin(lc_counter, lc_w_significant_peak):
                    ax[row, column].plot(
                        xplot,
                        sum_area_model_dict_bkgSubs_indiv_ids[id_clust][name_clust][energy_band][loop_model][lc_counter],
                        lw=4,
                        ls='-',
                        ms=0,
                        label='{}'.format(models[loop_model]),
                        color=color_model[loop_model],
                        rasterized=True
                    )
                    
                    ax[row, column].errorbar(rad_distrib_obs[f'{name_clust}_{energy_band}'].anulii[:-1], 
                                         rad_distrib_obs[f'{name_clust}_{energy_band}'].Nagn[:-1],
                                         yerr=rad_distrib_obs[f'{name_clust}_{energy_band}'].e_Nagn[:-1],
                                         marker='s',
                                         capsize=6,
                                         ms=8,
                                         alpha=0.75,
                                         color='k',
                                         label='Observations',
                                         rasterized=True
                                         )

                    ax[row, column].set_facecolor('lightgray')
                    ax[row, column].set_title('lc={}'.format(lc_counter), fontsize=30, weight='bold', color='tab:green')
    
        
                 
            ax[row, column].tick_params(axis='y', labelsize=15)
            ax[row, column].tick_params(axis='x', labelsize=15)
            
            if column != 0:
                ax[row, column].set(yticklabels=[])
            else:
                ax[row, column].set_ylabel('Number AGN', fontsize=28)
            if row < 9: 
                ax[row, column].set(xticklabels=[])
            else:
                ax[row, column].set_xlabel('r/R$_{500}$', fontsize=28)
        
            ax[row, column].set_xlim(0, 4)
            ax[row, column].set_ylim(-10, 30)
            ax[row, column].grid(True)
            
            lc_counter += 1
            if lc_counter > n_lc:
                break
    
    
    fig.suptitle(f'Cluster id={id_clust}, sensitivity map={name_clust.upper()}, band={energy_band}\n', fontsize=38)
    fig.tight_layout()
    
    plt.show()  

# Redshift distribution

In [ ]:
histo_z_lc_sample_dict = {}

for id_clst in lc_sample_list_csv:
    histo_z_lc_sample_dict.update( { f"{id_clst}":{} } )
    
    for name_clust in corename:
        histo_z_lc_sample_dict[id_clst].update( { f"{name_clust}":{} } )
        
        for energy_band in bands:
            histo_z_lc_sample_dict[id_clst][name_clust].update( { f"{energy_band}":{} } )
            
            for lc_list in lc_sample_list_csv[ id_lst[0] ]:
                if  lc_list == 'c_rand':
                    models = ['AMedian', 'GMedian', 'Gauss_mu-2.00_sigma0.50', 'Gauss_mu-1.50_sigma0.25', 'Gauss_mu-1.25_sigma0.10']
            
                elif lc_list == 'f_rand':
                    models = ['AMedian', 'GMedian']
                    
                else:
                    print("lc type non recognised")
                    break
                print(id_clst, name_clust, energy_band, lc_list)
                (sum_area_model_del, del_, hist_z_aux) = fun_sum_area._get_area_sample_lcs(
                    lc_sample_list_csv[ id_lst[0] ][lc_list][:], 
                    annulus_radius_dict[id_lst[0]], 
                    energy_band,
                    models, 
                    name_clust,
                    get_all_feat=True
                )
                
                histo_z_lc_sample_dict[id_clst][name_clust][energy_band].update({f"{lc_list}":hist_z_aux})

In [ ]:
with plt.style.context('default'):
    fig = plt.figure(figsize=(8, 5), dpi=100) 
    ax = fig.add_subplot(111)
    
    ls_lst = ['-', '--', '-.', '-', '--', '-.', '-', '--', '-.', '-', '--', '-.']
    
    random_state = 425
    rng = np.random.RandomState(random_state )
    lc_w_significant_peak_bkg = [ 14, 34, 51, 73, 69, 96, 89]
    
   
    jet = plt.get_cmap('cmr.pride') 
    values = np.arange(0, len(lc_w_significant_peak_bkg)+4, 1)
    cNorm  = mp_colors.Normalize(vmin=0, vmax=values[-1])
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
    
    range_z = np.arange(0, 3., .1)
    
    for loop, lc_num in enumerate(lc_w_significant_peak_bkg):
        colorVal = scalarMap.to_rgba(values[loop])
        print()
        for ring in np.arange(4, 5, 1):
            if ring == 4:
                alpha = 1
                lw = 2
            else:
                alpha = .3
                lw = 2
                
            xplot = (range_z[:-1] + range_z[1:])/2
            hist_z = np.array(histo_z_lc_sample_dict[id_lst[0]][name_clust][energy_band][lc_type])[0, lc_num, ring, :]  - \
                     np.mean(np.array( histo_z_lc_sample_dict[id_lst[0]][name_clust][energy_band][lc_type_f][0])[:, ring,:], axis=0)
            
            ax.plot(xplot, 
                    hist_z,
                    lw=3,
                    ls=ls_lst[loop],
                    label=f'lc = {lc_num}',
                    color=colorVal,
                    alpha=.75
                   )

    ax.legend(loc='center left', bbox_to_anchor=(1, .5), fontsize=16, ncol=1)
       
    ax.plot([0,20],[0,0],
             ls=':',
             lw=1.5,
             color='k')
    
    z_clust = 0.96
    ax.plot([z_clust, z_clust],[-5,20],
             ls=':',
             lw=1.5,
             color='k')
    
    ax.set_xlabel('z', fontsize=fs)
    ax.set_ylabel('N$_{AGN}$', fontsize=fs)
    
    ax.set_xlim(0, 2.5)
    ax.set_ylim(-1.5, 6)
    ax.tick_params(axis='both', labelsize=fs-8)
    
    ax.set_title(f'Ring 4 (2-2.5r$_{{500}}$)', fontsize=30)
    
    fig.tight_layout()
            
    plt.show()

# Infall population

In [ ]:
infall_id = pd.read_csv('./input/id_infall_pop_smMask_mDMmask.csv')
infall_id

In [ ]:
models_mod = [ 'AMedian-Gauss_mu-2.00_sigma0.50', 'AMedian-Gauss_mu-1.50_sigma0.25', 
               'AMedian-Gauss_mu-1.25_sigma0.10', 'GMedian-Gauss_mu-2.00_sigma0.50',
               'GMedian-Gauss_mu-1.50_sigma0.25', 'GMedian-Gauss_mu-1.25_sigma0.10' ]

In [ ]:
for id_obj in id_lst: 
    lc_type = 'c_rand'
    for lc in lc_sample_list_csv[f'{id_obj}']['c_rand']:
        mask_id_infall = np.isin(lc.id, infall_id)
        
        for new_model in models_mod:
            old_model = new_model.split('-')[0]
            mod_model = models_mod[0].split('-')[1] + '-' + models_mod[0].split('-')[2]
            
            for name_clust in corename:
                for energy_band in bands:
                    lc[f"area_{new_model}_{energy_band}_{name_clust}"] = \
                    lc[f"area_{old_model}_{energy_band}_{name_clust}"]
                    
                    lc[f"flux_pl{new_model}_{energy_band}"] = \
                    lc[f"flux_pl{old_model}_{energy_band}"]
                    
                    lc[f"lgLx{new_model}"] = lc[f"lgLx{old_model}"]
                    
                    lc.loc[mask_id_infall, f"area_{new_model}_{energy_band}_{name_clust}"] = \
                    lc[f"area_{mod_model}_{energy_band}_{name_clust}"][mask_id_infall ]
                    
                    lc.loc[mask_id_infall, f"flux_pl{new_model}_{energy_band}"] = \
                    lc[f"flux_pl{mod_model}_{energy_band}"][mask_id_infall ]
                    
                    lc.loc[mask_id_infall, f"lgLx{new_model}"] = \
                    lc[f"lgLx{mod_model}"][mask_id_infall ]


In [ ]:
sum_area_model_dict_ids_mod = {}

for id_obj in id_lst: 
    print(f'{id_obj}')
    sum_area_model_dict_ids_mod.update(
        {
         f'{id_obj}': fun_sum_area.sum_area_ids(
                                   corename,
                                   bands, 
                                   lc_sample_list_csv[f'{id_obj}'], 
                                   annulus_radius_dict[id_obj], 
                                   models_mod, 
                                   models_f=models_f
                    )
        }
)

In [ ]:
with plt.style.context('default'):
    fig = plt.figure(figsize=(6, 5), dpi=150) 
    ax = fig.add_subplot(111)
    
    xplot = (np.arange(0, 7.5, .5)[1:] + np.arange(0, 7.5, .5)[:-1])*.5
    
    color_model = ['tab:orange', 'tab:green']
    markers = ['o', 's', '^']
    model_names  = ['Model 1', 'Model 2']
    jet = cm = plt.get_cmap('plasma') 
    values = np.arange(0, 10, 1)
    cNorm  = mp_colors.Normalize(vmin=0, vmax=values[-1])
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
    
    ax.fill_between([], [],
               alpha=0.5,
               color='k',
               label='68% c.i.'
                )
    
    id_clust = id_lst[0]
    name_clust = 'pl26' 
    energy_band = 'full'
    lc_type = 'c_rand'
    lc_type_f = 'f_rand'
    
    ax.errorbar(rad_distrib_obs[f'{name_clust}_{energy_band}'].anulii[:-1], 
                rad_distrib_obs[f'{name_clust}_{energy_band}'].Nagn[:-1],
                yerr=rad_distrib_obs[f'{name_clust}_{energy_band}'].e_Nagn[:-1],
                marker='s',
                capsize=6,
                ms=8,
                color='k',
                label='Observations'
               )
    
    loop_model = 1
    sum_area_loop = sum_area_model_dict_ids[id_clust][name_clust][energy_band][lc_type][loop_model]
    
    ax.fill_between(xplot[:8],
                    oneSigmaLow_sum_area_model_dict_bkgSubs_ids[id_clust][f'{name_clust}'][f'{energy_band}'][loop_model][:8],
                    oneSigmaUp_sum_area_model_dict_bkgSubs_ids[id_clust][f'{name_clust}'][f'{energy_band}'][loop_model][:8],
                    color=color_model[loop_model],
                    alpha=.25)
    
    ax.plot(xplot[:8], 
            sum_area_model_dict_bkgSubs_ids[id_clust][f'{name_clust}'][f'{energy_band}'][loop_model][:8],
            marker=markers[loop_model],
            lw=3,
            ms=0,
            color=color_model[loop_model],
            label=f'{model_names[loop_model]}',
            )
    
    loop_model = 4#5
    sum_area_loop = sum_area_model_dict_ids_mod[id_clust][name_clust][energy_band][lc_type][loop_model]

    loop_model_f = 1
    
    up_lim  = []   
    low_lim = []
    
    for annuli_loop, annuli in enumerate(xplot):
        up_lim.append(np.quantile(np.array(sum_area_loop)[:,annuli_loop] -
                                  np.mean(sum_area_field_model_suffle_dict_ids[id_clust][name_clust][energy_band][lc_type_f][loop_model_f], axis=0)[annuli_loop], .84))
        low_lim.append(np.quantile(np.array(sum_area_loop)[:,annuli_loop] -
                               np.mean(sum_area_field_model_suffle_dict_ids[id_clust][name_clust][energy_band][lc_type_f][loop_model_f], axis=0)[annuli_loop], .16))
    low_lim_model = low_lim
    up_lim_model = up_lim 
        
    ax.fill_between(xplot[:8],
                    low_lim_model[:8],
                    up_lim_model[:8],
                    facecolor="none",
                    edgecolor='magenta',
                    hatch='x',
                    linewidth=0.2,
                    alpha=1)
    
    label = 'Gaussian: $\mu$=-1.25, $\sigma$=0.1'
    ax.plot(xplot[:8], 
            (np.mean(sum_area_loop, axis=0) - 
             np.mean(sum_area_field_model_suffle_dict_ids[id_clust][name_clust][energy_band][lc_type_f][loop_model_f], axis=0))[:8],
             marker='o',
             lw=3,
             ms=0,
             color='magenta',
             label=f'{label}',
            )
        
    ax.plot([0,5],[0,0],
           ls=':',
           lw=1.5,
           color='k')
    
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    
    
    ax.set_xlabel('r/R$_{500}$', fontsize=16)
    ax.set_ylabel('Number AGN', fontsize=16)
    
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    
    ax.legend()
    
    ax.set_xlim(0, 4)
    ax.grid(True)
    ax.set_title(f'Cluster id={id_clust}', fontsize=16)
    
    fig.tight_layout()
    plt.show()